In [2]:
import json
import numpy as np
from pathlib import Path
import sys
import os
import yaml
import pandas as pd
import pickle

import xobjects as xo
import xtrack as xt
import xpart as xp
import xcoll as xc
import scipy
import io 

from IPython import embed





# ---------------------------- LOADING FUNCTIONS ----------------------------


def load_colldb_new(filename):
    with open(filename, "r") as infile:
        coll_data_string = ""
        family_settings = {}
        family_types = {}
        onesided = {}
        tilted = {}
        bend = {}
        xdim = {}
        ydim = {}

        for l_no, line in enumerate(infile):
            if line.startswith("#"):
                continue  # Comment
            if len(line.strip()) == 0:
                continue  # Empty line
            sline = line.split()
            if len(sline) < 6 or sline[0].lower() == "crystal" or sline[0].lower() == "target":
                if sline[0].lower() == "nsig_fam":
                    family_settings[sline[1]] = sline[2]
                    family_types[sline[1]] = sline[3]
                elif sline[0].lower() == "onesided":
                    onesided[sline[1]] = int(sline[2])
                elif sline[0].lower() == "tilted":
                    tilted[sline[1]] = [float(sline[2]), float(sline[3])]
                elif sline[0].lower() == "crystal":
                    bend[sline[1]] = float(sline[2])
                    xdim[sline[1]] = float(sline[3])
                    ydim[sline[1]] = float(sline[4])
                elif sline[0].lower() == "target":
                    xdim[sline[1]] = float(sline[2])
                    ydim[sline[1]] = float(sline[3])
                elif sline[0].lower() == "settings":
                    pass  # Acknowledge and ignore this line
                else:
                    raise ValueError(f"Unknown setting {line}")
            else:
                coll_data_string += line

    names = ["name", "opening", "material", "length", "angle", "offset"]

    df = pd.read_csv(io.StringIO(coll_data_string), delim_whitespace=True,
                     index_col=False, skip_blank_lines=True, names=names)

    df["angle"] = df["angle"] 
    df["name"] = df["name"].str.lower() # Make the names lowercase for easy processing
    df["gap"] = df["opening"].apply(lambda s: float(family_settings.get(s, s)))
    df["type"] = df["opening"].apply(lambda s: family_types.get(s, "UNKNOWN"))
    df["side"] = df["name"].apply(lambda s: onesided.get(s, 0))
    df["bend"] = df["name"].apply(lambda s: bend.get(s, 0))
    df["xdim"] = df["name"].apply(lambda s: xdim.get(s, 0))
    df["ydim"] = df["name"].apply(lambda s: ydim.get(s, 0))
    df["tilt_left"] = df["name"].apply(lambda s: np.deg2rad(tilted.get(s, [0, 0])[0]))
    df["tilt_right"] = df["name"].apply(lambda s: np.deg2rad(tilted.get(s, [0, 0])[1]))
    df = df.set_index("name").T

    # Ensure the collimators marked as one-sided or tilted are actually defined
    defined_set = set(df.columns) # The data fram was transposed so columns are names
    onesided_set = set(onesided.keys())
    tilted_set = set(tilted.keys())
    if not onesided_set.issubset(defined_set):
        different = onesided_set - defined_set
        raise SystemExit('One-sided collimators not defined: {}'.format(", ".join(different)))
    if not tilted_set.issubset(defined_set):
        different = tilted_set - defined_set
        raise SystemExit('Tilted collimators not defined: {}'.format(",".join(different)))
    return df.T


def find_axis_intercepts(x_coords, y_coords):
    x_intercepts = []
    y_intercepts = []

    for i in range(len(x_coords)):
        x1, y1 = x_coords[i], y_coords[i]
        x2, y2 = x_coords[(i + 1) % len(x_coords)], y_coords[(i + 1) % len(y_coords)]

        if x1 == x2:
        # Vertical line, no y-intercept
            y_intercept = 0.0 if x1 == x2 == 0.0 else None
        else:
            slope = (y2 - y1) / (x2 - x1)
            y_intercept = y1 - (slope * x1)

        if y1 == y2:
        # Horizontal line, no x-intercept
            x_intercept = 0.0 if y1 == y2 == 0.0 else None
        else:
            slope = (x2 - x1) / (y2 - y1)
            x_intercept = x1 - (slope * y1)

        # Check if the x-intercept is within the range of x1 and x2
        if x_intercept is not None and (x1 <= x_intercept <= x2 or x2 <= x_intercept <= x1):
            x_intercepts.append(x_intercept)

        # Check if the y-intercept is within the range of y1 and y2
        if y_intercept is not None and (y1 <= y_intercept <= y2 or y2 <= y_intercept <= y1):
            y_intercepts.append(y_intercept)

    return x_intercepts, y_intercepts



def find_bad_offset_apertures(line):
    aperture_offsets = {}
    for name, element in line.element_dict.items():
        if 'offset' in name and element.__class__.__name__.startswith('XYShift'):
            aper_name = name.split('_offset')[0]
            aperture_offsets[aper_name] = (element.dx, element.dy)

    bad_apers = {}
    for ap_name, offset in aperture_offsets.items():
        aperture_el = line.element_dict[ap_name]

        cname= aperture_el.__class__.__name__
        ap_dict = aperture_el.to_dict()

        if cname == 'LimitEllipse':
            x_min = -ap_dict['a']
            x_max = ap_dict['a']
            y_min = -ap_dict['b']
            y_max = ap_dict['b']
        elif cname == 'LimitRect':
            x_min = ap_dict['min_x']
            x_max = ap_dict['max_x']
            y_min = ap_dict['min_y']
            y_max = ap_dict['max_y']
        elif cname == 'LimitRectEllipse':
            x_min = -ap_dict['max_x']
            x_max = ap_dict['max_x']
            y_min = -ap_dict['max_y']
            y_max = ap_dict['max_y']
        elif cname == 'LimitRacetrack':
            x_min = ap_dict['min_x']
            x_max = ap_dict['max_x']
            y_min = ap_dict['min_y']
            y_max = ap_dict['max_y']
        elif cname == 'LimitPolygon':
            x_intercepts, y_intercepts = find_axis_intercepts(ap_dict['x_vertices'],
                                                            ap_dict['y_vertices'])
            x_min = min(x_intercepts)
            x_max = max(x_intercepts)
            y_min = min(y_intercepts)
            y_max = max(y_intercepts)

        tolerance = 5e-3
        """if (x_max - offset[0] < tolerance 
            or -x_min + offset[0] < tolerance 
            or y_max - offset[1] < tolerance 
            or -y_min + offset[1] < tolerance):"""
        if (offset[0] -x_max > tolerance 
            or  -offset[0] + x_min > tolerance 
            or  offset[1] - y_max > tolerance 
            or  -offset[1] + y_min > tolerance ):
                bad_apers[ap_name] = (x_min, x_max, y_min, y_max, offset[0], offset[1])

    return bad_apers





In [22]:
# ---------------------------- MAIN ----------------------------


config_file = 'config_sim.yaml'

with open(config_file, 'r') as stream:
    config_dict = yaml.safe_load(stream)

# Configure run parameters
run_dict = config_dict['run']

beam          = run_dict['beam']
plane         = run_dict['plane']

num_turns     = run_dict['turns']
num_particles = run_dict['nparticles']
engine        = run_dict['engine']

seed          = run_dict['seed']

TCCS_align_angle_step = float(run_dict['TCCS_align_angle_step'])

normalized_emittance = run_dict['normalized_emittance']

mode = run_dict['mode']
print('\nMode: ', mode, '\n', 'Seed: ', seed, '\n')

save_list = run_dict['save_list']

# Setup input files
file_dict = config_dict['input_files']

coll_file = os.path.expandvars(file_dict['collimators'])
line_file = os.path.expandvars(file_dict[f'line_b{beam}'])

print('Input files:\n', line_file, '\n', coll_file, '\n')

if coll_file.endswith('.yaml'):
    with open(coll_file, 'r') as stream:
        coll_dict = yaml.safe_load(stream)['collimators']['b'+config_dict['run']['beam']]
if coll_file.endswith('.data'):
    coll_dict = load_colldb_new(coll_file).to_dict('index')

TCCS_gap = float(run_dict['TCCS_gap'])
TCCP_gap = float(run_dict['TCCP_gap'])
TARGET_gap = float(run_dict['TARGET_gap'])
PIXEL_gap = float(run_dict['PIXEL_gap'])


print('\nTwocryst gaps:   TCCS: ', TCCS_gap,', TARGET: ', TARGET_gap , ', TCCP:' ,TCCP_gap ,', PIXEL:' , PIXEL_gap, '\n')

context = xo.ContextCpu(omp_num_threads='auto')

# Define output path
path_out = Path.cwd() / 'Outputdata'

if not path_out.exists():
    os.makedirs(path_out)



# ---------------------------- SETUP LINE ----------------------------

# Load from json
line = xt.Line.from_json(line_file)

end_s = line.get_length()

TCCS_name = 'tccs.5r3.b2'
TCCP_name = 'tccp.4l3.b2'
TARGET_name = 'target.4l3.b2'
PIXEL_name = 'pixel.detector'
TCP_name = 'tcp.d6r7.b2'

d_pix = 1 # [m]

TCCS_loc = end_s - 6773.7 #6775
TCCP_loc = end_s - 6653.3 #6655
TARGET_loc = end_s - (6653.3 + coll_dict[TCCP_name]["length"]/2 + coll_dict[TARGET_name]["length"]/2+0.0001)
PIXEL_loc = end_s - (6653.3 - coll_dict[TCCP_name]["length"]/2 - d_pix)
TCP_loc = line.get_s_position()[line.element_names.index(TCP_name)]


line.insert_element(at_s=TCCS_loc, element=xt.Marker(), name=TCCS_name)
line.insert_element(at_s=TCCS_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name=TCCS_name+'_aper')
line.insert_element(at_s=TCCP_loc, element=xt.Marker(), name=TCCP_name)
line.insert_element(at_s=TCCP_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name=TCCP_name+'_aper')
line.insert_element(at_s=TARGET_loc, element=xt.Marker(), name=TARGET_name)
line.insert_element(at_s=TARGET_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name= TARGET_name + '_aper')
line.insert_element(at_s=PIXEL_loc, element=xt.Marker(), name=PIXEL_name)
#line.insert_element(at_s=PIXEL_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name= PIXEL_name + '_aper')


TCCS_monitor = xt.ParticlesMonitor(num_particles=num_particles, start_at_turn=0, stop_at_turn=num_turns)
TARGET_monitor = xt.ParticlesMonitor(num_particles=num_particles, start_at_turn=0, stop_at_turn=num_turns)
PIXEL_monitor = xt.ParticlesMonitor(num_particles=num_particles, start_at_turn=0, stop_at_turn=num_turns)
TCP_monitor = xt.ParticlesMonitor(num_particles=num_particles, start_at_turn=0, stop_at_turn=num_turns)
dx = 1e-11
line.insert_element(at_s = TCCS_loc - coll_dict[TCCS_name]["length"]/2 - dx, element=TCCS_monitor, name='TCCS_monitor')
line.insert_element(at_s = TARGET_loc - coll_dict[TARGET_name]["length"]/2 - dx, element=TARGET_monitor, name='TARGET_monitor')
line.insert_element(at_s = PIXEL_loc, element=PIXEL_monitor, name='PIXEL_monitor')
#line.insert_element(at_s = TCP_loc + coll_dict[TCP_name]/2 + dx, element=TCP_monitor, name='TCP_monitor')


bad_aper = find_bad_offset_apertures(line)
print('Bad apertures : ', bad_aper)
print('Replace bad apertures with Marker')
for name in bad_aper.keys():
    line.element_dict[name] = xt.Marker()
    print(name, line.get_s_position(name), line.element_dict[name])

# Aperture model check
print('\nAperture model check on imported model:')
df_imported = line.check_aperture()
assert not np.any(df_imported.has_aperture_problem)


# Initialise collmanager
if coll_file.endswith('.yaml'):
    coll_manager = xc.CollimatorManager.from_yaml(coll_file, line=line, beam=beam, _context=context, ignore_crystals=False)
elif coll_file.endswith('.data'):
    coll_manager = xc.CollimatorManager.from_SixTrack(coll_file, line=line, _context=context, ignore_crystals=False, nemitt_x = normalized_emittance,  nemitt_y = normalized_emittance, beam=beam)
    # switch on cavities
    speed = line.particle_ref._xobject.beta0[0]*scipy.constants.c
    harmonic_number = 35640
    voltage = 12e6/len(line.get_elements_of_type(xt.Cavity)[1])
    frequency = harmonic_number * speed /line.get_length()
    for side in ['l', 'r']:
        for cell in ['a','b','c','d']:
            line[f'acsca.{cell}5{side}4.b2'].voltage = voltage
            line[f'acsca.{cell}5{side}4.b2'].frequency = frequency


line.to_json('./input_files/HL_twocryst.json')


Mode:  target_absorber 
 Seed:  12345 

Input files:
 /afs/cern.ch/work/c/cmaccani/xsuite_sim/twocryst_sim/input_files/HL_IR7_tune_changed/b4_sequence_patched_tune.json 
 /afs/cern.ch/work/c/cmaccani/xsuite_sim/twocryst_sim/input_files/colldbs/CollDB_HL_tight_b4.data 


Twocryst gaps:   TCCS:  7.2 , TARGET:  33.6 , TCCP: 33.6 , PIXEL: 33.6 



Loading line from dict:   0%|          | 0/151124 [00:00<?, ?it/s]

Done loading line from dict.           


: 

In [ ]:
# Install collimators into line
if engine == 'everest':
    coll_names = coll_manager.collimator_names

    if mode == 'target_absorber': 
        black_absorbers = [TARGET_name,]
    else: 
        black_absorbers = []

    everest_colls = [name for name in coll_names if name not in black_absorbers]

    coll_manager.install_everest_collimators(names=everest_colls,verbose=True)
    coll_manager.install_black_absorbers(names = black_absorbers, verbose=True)
else:
    raise ValueError(f"Unknown scattering engine {engine}!")


# Aperture model check
print('\nAperture model check after introducing collimators:')
df_with_coll = line.check_aperture()
assert not np.any(df_with_coll.has_aperture_problem)

    
# Build the tracker
#coll_manager.build_tracker()
coll_manager.build_tracker()

# Set the collimator openings based on the colldb,
# or manually override with the option gaps={collname: gap}
#coll_manager.set_openings()
coll_manager.set_openings(gaps = {TCCS_name: TCCS_gap, TCCP_name: TCCP_gap, TARGET_name: TARGET_gap})


print("\nTCCS aligned to beam: ", line[TCCS_name].align_angle)
#line[TTCS_name].align_angle = TTCS_align_angle_step
print("TCCS align angle incremented by step: ", TCCS_align_angle_step)
line[TCCS_name].align_angle = line[TCCS_name].align_angle + TCCS_align_angle_step
print("TCCS final alignment angle: ", line[TCCS_name].align_angle)


# Aperture model check
print('\nAperture model check after introducing collimators:')
df_with_coll = line.check_aperture()
assert not np.any(df_with_coll.has_aperture_problem)

# Printout useful informations
idx_TCCS = line.element_names.index(TCCS_name)
idx_TARGET = line.element_names.index(TARGET_name)
idx_TCCP = line.element_names.index(TCCP_name)
idx_PIXEL = line.element_names.index(PIXEL_name)

tw = line.twiss()
beta_y_TCCS = tw[:,TCCS_name]['bety'][0]
beta_y_TCCP = tw[:,TCCP_name]['bety'][0]
beta_y_TARGET = tw[:,TARGET_name]['bety'][0]
beta_y_PIXEL = tw[:,PIXEL_name]['bety'][0]
beta_rel = line.particle_ref._xobject.beta0[0]
gamma = line.particle_ref._xobject.gamma0[0]

emittance_phy = normalized_emittance/(beta_rel*gamma)

sigma_TCCS = np.sqrt(emittance_phy*beta_y_TCCS)
sigma_TCCP = np.sqrt(emittance_phy*beta_y_TCCP)
sigma_TARGET = np.sqrt(emittance_phy*beta_y_TARGET)
sigma_PIXEL = np.sqrt(emittance_phy*beta_y_PIXEL)

print(f"\nTCCS\nCrystalAnalysis(n_sigma={line.elements[idx_TCCS].jaw_L/sigma_TCCS}, length={ coll_dict[ TCCS_name]['length']}, ydim={ coll_dict[ TCCS_name]['xdim']}, xdim={ coll_dict[ TCCS_name]['ydim']}, bend={ coll_dict[ TCCS_name]['bend']}, align_angle={ line.elements[idx_TCCS].align_angle}, sigma={sigma_TCCS})")
print(f"TARGET\nTargetAnalysis(n_sigma={line.elements[idx_TARGET].jaw_L/sigma_TARGET}, length={ coll_dict[ TARGET_name]['length']}, ydim={ coll_dict[ TARGET_name]['xdim']}, xdim={ coll_dict[ TARGET_name]['ydim']}, sigma={sigma_TARGET})")
print(f"TCCP\nCrystalAnalysis(n_sigma={line.elements[idx_TCCP].jaw_L/sigma_TCCP}, length={ coll_dict[ TCCP_name]['length']}, ydim={ coll_dict[ TCCP_name]['xdim']}, xdim={ coll_dict[ TCCP_name]['ydim']}, bend={ coll_dict[ TCCP_name]['bend']}, sigma={sigma_TCCP})")
print(f"PIXEL\nTargetAnalysis(n_sigma={line.elements[idx_TCCP].jaw_L/sigma_TCCP}, length={0}, ydim={0}, xdim={0}, sigma={sigma_PIXEL})\n")

In [18]:
line[TCP_name]

EverestCollimator(inactive_front=0.0, active_length=0.6, inactive_back=0.0, jaw_L=0.0022989302491263903, jaw_R=-0.0022989302491263903, ref_x=4.910024205232376e-06, ref_y=8.480175699409359e-07, sin_zL=1.0, cos_zL=6.123233995736766e-17, sin_zR=1.0, cos_zR=6.123233995736766e-17, sin_yL=0.0, cos_yL=1.0, tan_yL=1.0, sin_yR=0.0, cos_yR=1.0, tan_yR=1.0, _side=0, active=1, _internal_record_id=RecordIdentifier(buffer_id=0, offset=0), _material=Material(Z=6.65, A=13.53, density=2.5, excitation_energy=8.71e-08, nuclear_radius=0.25, nuclear_elastic_slope=76.7, cross_section=[3.62e-01 2.47e-01 0.00e+00 0.00e+00 0.00e+00 9.40e-05], hcut=0.02, name=MolybdenumGraphite, radiation_length=0.1193), rutherford_rng=RandomRutherford(lower_val=0.0009982, upper_val=0.02, A=0.0012280392539122623, B=53.50625, Newton_iterations=7), _tracking=0)

In [19]:
line[TCCS_name]

EverestCrystal(inactive_front=0.0, active_length=0.004, inactive_back=0.0, jaw_L=0.002023846744961569, jaw_R=-0.025, ref_x=-3.8140465516383823e-06, ref_y=-3.550398552456167e-08, sin_zL=1.0, cos_zL=6.123233995736766e-17, sin_zR=1.0, cos_zR=6.123233995736766e-17, sin_yL=0.0, cos_yL=1.0, tan_yL=1.0, sin_yR=0.0, cos_yR=1.0, tan_yR=1.0, _side=1, active=1, _internal_record_id=RecordIdentifier(buffer_id=0, offset=0), align_angle=-1.6117913846552086e-05, bend=80.0, xdim=0.002, ydim=0.035, thick=0.0, miscut=0.0, _orient=1, _material=CrystalMaterial(Z=14.0, A=28.08, density=2.33, excitation_energy=1.73e-07, nuclear_radius=0.441, nuclear_elastic_slope=120.14, cross_section=[6.64e-01 4.30e-01 0.00e+00 0.00e+00 0.00e+00 3.90e-04], hcut=0.02, name=Silicon, crystal_radiation_length=0.0937, crystal_nuclear_length=0.4652, crystal_plane_distance=9.6e-08, crystal_potential=21.34, nuclear_collision_length=0.3016), rutherford_rng=RandomRutherford(lower_val=0.0009982, upper_val=0.02, A=0.0016160247264725453

In [20]:
line.to_json('HL_twocryst.json')

In [21]:
new_line = xt.Line.from_json('HL_twocryst.json')

Loading line from dict:   0%|          | 0/150839 [00:00<?, ?it/s]

KeyError: 'EverestCollimator'